In [ ]:
# default_exp pairwise_distance

# Pairwise Distances

> Functions to calculate the pairwise distances between vectors

In [ ]:
# export
import numpy as np
import numba as nb
from numba import cuda
from scipy.spatial.distance import pdist

In [ ]:
# hide
@nb.njit
def pairwise_numba(X, is_float_type):
    m = X.shape[0]
    n = X.shape[1]

    D = np.empty((int(m * (m - 1) / 2), 1), dtype=np.float64)  # corrected dtype
    ind = 0

    for i in range(m):
        for j in range(i+1, m):
            d = 0.0

            for k in range(n):
                if is_float_type[k]:
                    tmp = X[i, k] - X[j, k]
                    d += tmp * tmp
                else:
                    if X[i, k] != X[j, k]:
                        d += 1.

            D[ind] = np.sqrt(d)
            ind += 1

    return D.reshape(1, -1)[0]

In [ ]:
# hide
vecs = np.random.randn(3,4)

In [ ]:
# hide
dists = pairwise_numba(vecs, np.ones(vecs.shape[1],dtype=bool))

In [ ]:
# export

USE_64 = False

if USE_64:
    bits = 64
    np_type = np.float64
else:
    bits = 32
    np_type = np.float32

In [ ]:
# export
@cuda.jit("void(float{}[:, :], float{}[:, :])".format(bits, bits))
def _euclidian_distance_matrix(mat, out):
    "CUDA kernel used to calculate the squared euclidian distance between all rows in a matrix"
    m = mat.shape[0]
    n = mat.shape[1]
    i, j = cuda.grid(2)
    d = 0
    if i < m and j > i and j < m:
        # calculate ||x - y||^2
        for k in range(n):
            tmp = mat[i, k] - mat[j, k]
            d += tmp * tmp
        out[i, j] = d

In [ ]:
# export
def gpu_dist_matrix(mat):
    "calculate the squared euclidian distance between all pairs of rows in a given matrix"
    rows = mat.shape[0]

    block_dim = (16, 16)
    grid_dim = (int(rows/block_dim[0] + 1), int(rows/block_dim[1] + 1))

    stream = cuda.stream()
    mat2 = cuda.to_device(np.asarray(mat, dtype=np_type), stream=stream)
    out2 = cuda.device_array((rows, rows))
    _euclidian_distance_matrix[grid_dim, block_dim](mat2, out2)
    out = out2.copy_to_host(stream=stream)

    return out

In [ ]:
# hide
dists_gpu = gpu_dist_matrix(vecs)

In [ ]:
# hide
dists**2

array([9.89992139, 2.31393036, 7.07580765])

In [ ]:
# hide
dists_gpu

array([[0.        , 9.89991954, 2.31392908],
       [0.        , 0.        , 7.07580663],
       [0.        , 0.        , 0.        ]])

In [ ]:
# hide
np.allclose(dists_gpu[np.where(np.triu(dists_gpu,1) != 0)],dists**2)

True

In [ ]:
# export

@cuda.jit("void(float{}[:, :], float{}[:, :], float{}[:, :])".format(bits, bits, bits))
def _pairwise_distance_matrix(compMat, mixMat, out):
    "CUDA kernel used to calcualte squared euclidian distance  between pairs of rows in two matrices"
    nC = compMat.shape[0]
    nM = mixMat.shape[0]
    dim = compMat.shape[1]
    i, j = cuda.grid(2)
    d = 0
    if i < nC and j < nM:
        # calculate ||c_i - m_j||^2
        for k in range(dim):
            tmp = compMat[i, k] - mixMat[j, k]
            d += tmp * tmp
        out[i, j] = d

In [ ]:
# export

def component_mixture_dist_matrix(compMat, mixMat):
    "calculate the squared euclidian distance between pairs of rows in the two given matrices"
    compRows = compMat.shape[0]
    mixRows = mixMat.shape[0]

    block_dim = (16, 16)
    grid_dim = (int(compRows/block_dim[0] + 1), int(mixRows/block_dim[1] + 1))

    stream = cuda.stream()
    compMat2 = cuda.to_device(np.asarray(compMat, dtype=np_type), stream=stream)
    mixMat2 = cuda.to_device(np.asarray(mixMat, dtype=np_type), stream=stream)
    out2 = cuda.device_array((compRows, mixRows))
    _pairwise_distance_matrix[grid_dim, block_dim](compMat2, mixMat2, out2)
    out = out2.copy_to_host(stream=stream)

    return out

In [ ]:
# hide
compMat = np.array([[1,2,3],
                    [4,5,6],
                    [7,8,9]])

mixMat = np.array([[11,21,31],
                   [41,51,61]])

In [ ]:
# hide
puDists = component_mixture_dist_matrix(compMat, mixMat)

In [ ]:
# hide
puDists

array([[1245.00026172, 7365.00106657],
       [ 930.00013048, 6510.00106498],
       [ 669.00012999, 5709.00106349]])

In [ ]:
# hide
out = np.zeros((compMat.shape[0], mixMat.shape[0]))
for i in range(compMat.shape[0]):
    for j in range(mixMat.shape[0]):
        out[i,j] = np.linalg.norm(compMat[i] - mixMat[j],ord=2)**2

In [ ]:
# hide
np.allclose(puDists,out)

True